In [1]:
import os
join = os.path.join
import numpy as np
from glob import glob
import torch
from segment_anything.build_sam3D import sam_model_registry3D
from segment_anything.utils.transforms3D import ResizeLongestSide3D
from segment_anything import sam_model_registry
from tqdm import tqdm
import argparse
import SimpleITK as sitk
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchio as tio
import numpy as np
from collections import OrderedDict, defaultdict
import json
import pickle
from utils.click_method import get_next_click3D_torch_ritm, get_next_click3D_torch_2
import utils.data_loader 
import einops as E
import torch
Dataset_Union_ALL_Val = utils.data_loader.Dataset_Union_ALL_Val


In [19]:
# hardcode for juptyer notebook compatability

args = argparse.Namespace(
    test_data_path = '/home/t722s/Desktop/Datasets/BratsTestData/',
    checkpoint_path=  '',
    vis_path = '/home/t722s/Desktop/Sam-Med3DTest/segmentation_maps',
    eval_dir='/home/t722s/Desktop/Sam-Med3DTest/evalBrats/',
    interactive=True,
    model='SAM-Med3D',
    device = 'cuda',
    num_clicks = 5,
    seed=2023,
    dim = 3,
    crop_size = 128,
    data_type = 'Ts',
    split_num = 1,
    split_idx = 0,
)


In [22]:
def finetune_model_predict3D(img3D, gt3D, sam_model_tune, device='cuda', interactive=True, points_list = None, num_clicks=5, prev_masks=None):
    norm_transform = tio.ZNormalization(masking_method=lambda x: x > 0)
    img3D = norm_transform(img3D.squeeze(dim=1)) # (N, C, W, H, D)
    img3D = img3D.unsqueeze(dim=1)

    click_points = []
    click_labels = []

    pred_list = []

    iou_list = []
    dice_list = []
    if prev_masks is None:
        prev_masks = torch.zeros_like(gt3D).to(device)
    low_res_masks = F.interpolate(prev_masks.float(), size=(args.crop_size//4,args.crop_size//4,args.crop_size//4))

    for num_click in range(num_clicks):
        with torch.no_grad():
            # Modified by Tim
            if(True): #(num_click==0 | interactive == False):
                batch_points, batch_labels = points_list[num_click], torch.tensor([1])
                print(f'Testing: points_list: {points_list}')
            #else:
            batch_points, batch_labels = get_next_click3D_torch_ritm(prev_masks.to(device), gt3D.to(device))
            print(f'Testing click_method: {batch_points, batch_labels}')

                
            points_co = torch.cat(batch_points, dim=0).to(device)  
            points_la = torch.cat(batch_labels, dim=0).to(device)  

            click_points.append(points_co)
            click_labels.append(points_la)

            points_input = points_co
            labels_input = points_la


In [37]:
from importlib import reload
reload(utils.data_loader)
Dataset_Union_ALL_Val = utils.data_loader.Dataset_Union_ALL_Val

In [42]:
def f():
    # Added by Tim
    points_list = np.array([[1],[2],[3]])
    print(f'Before import numpy: {points_list}')
    print(f'Before import torch: {torch.tensor(points_list)}')
    points_mask = np.zeros(shape = (tio_image.shape[3], tio_image.shape[2], tio_image.shape[1])) # Skip the color channel for now; reintroduce in label_map definition. Reverse order since sitk uses WHD while numpy uses DHW 
    points_mask[*points_list.T] = 1
    points_mask = E.rearrange(points_mask, pattern = 'x y z -> z y x') # Rearrange back to sitk WHD
    points_mask = tio.LabelMap(tensor = torch.from_numpy(points_mask).float().unsqueeze(0), affine = tio_image.affine)

    subject = tio.Subject(
        image = tio_image,
        points_mask = points_mask,
        label = tio.LabelMap.from_sitk(sitk_label)
    )

    if '/ct_' in self.image_paths[index]:
        subject = tio.Clamp(-1000,1000)(subject)

    if self.transform:
        try:
            subject = self.transform(subject)
        except:
            print(self.image_paths[index])
    
    points_list = 0

    return subject.image.data.clone().detach(), subject.label.data.clone().detach(), torch.tensor(points_list), self.image_paths[index] # Later don't return label data

tensor(5)

In [38]:

infer_transform = [
    tio.ToCanonical(),
    tio.CropOrPad(mask_name='points_mask', target_shape=(args.crop_size,args.crop_size,args.crop_size)), # Will center the cropping/padding at the center of the bounding box of the clicks supplied.
]

test_dataset = Dataset_Union_ALL_Val(
    paths=args.test_data_path, 
    points_path=os.path.join(args.eval_dir, 'prompts.pkl'),
    dim=args.dim,
    mode="Val", 
    data_type=args.data_type, 
    transform=tio.Compose(infer_transform),
    split_num=args.split_num,
    split_idx=args.split_idx,
    pcc=False,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    sampler=None,
    batch_size=1, 
    shuffle=True
)

# Load in model
checkpoint_path = args.checkpoint_path

device = args.device
print("device:", device)


# Initialise results storage variables and directories
out_dice_all = OrderedDict()

vis_root = args.vis_path
os.makedirs(vis_root, exist_ok=True)

# Perform inference
for batch_data in tqdm(test_dataloader):
    image3D, gt3D, points_list, img_name = batch_data
    print(f'After import: {points_list}')
    sz = image3D.size()
    if(sz[2]<args.crop_size or sz[3]<args.crop_size or sz[4]<args.crop_size):
        print("[ERROR] wrong size", sz, "for", img_name)
    
    if(args.dim==3):
        seg_mask_list, points, labels, iou_list, dice_list = finetune_model_predict3D(
            image3D, gt3D, sam_model_tune = None, device=device, 
            interactive=args.interactive, points_list = points_list, num_clicks=args.num_clicks, 
            prev_masks=None)

device: cuda


  0%|          | 0/1 [00:00<?, ?it/s]

Before import numpy: [[73 75 67]
 [61 46 71]
 [59 47 72]
 [54 56 67]
 [69 62 54]]
Before import torch: tensor([[73, 75, 67],
        [61, 46, 71],
        [59, 47, 72],
        [54, 56, 67],
        [69, 62, 54]])
After import: tensor([0])
Testing: points_list: tensor([0])


  0%|          | 0/1 [00:00<?, ?it/s]

Testing click_method: ([tensor([[[69, 58, 71]]])], [tensor([[1]])])


IndexError: index 1 is out of bounds for dimension 0 with size 1